1). Loading library

In [3]:
import pandas as pd
import numpy as np
import matplotlib
import sklearn as skl
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgbm
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score,accuracy_score
import warnings
warnings.filterwarnings('ignore')

In [24]:
train=pd.read_csv('NigeriaAutoInsurance_Vehicle.csv')
StateName=pd.read_csv('NigerianStateNames.csv')

In [26]:
train['target'].value_counts()

0    2167
1     500
Name: target, dtype: int64

3). Exploring and transforming data

In [42]:
train.head()

,ID,Policy Start Date,Policy End Date,Gender,Age,First Transaction Date,No_Pol,Car_Category,Subject_Car_Colour,Subject_Car_Make,LGA_Name,State,ProductName,target
0,ID_00N7XT1,2010-07-06,2011-07-05,Female,71,2010-07-06,2,JEEP,Black,REXTON,Badagry,Benue,Car Classic,0
1,ID_01ZY7FF,2010-08-07,2011-08-06,Male,30,2010-08-07,1,Saloon,Grey,TOYOTA,Ikeja,Lagos,Car Classic,1
2,ID_022U6TV,2010-10-28,2011-10-27,Male,40,2010-10-28,2,Saloon,Black,Honda,Abuja Municipal,Abuja-Municipal,Car Classic,0
3,ID_02DJWEN,2010-01-23,2011-01-16,Male,41,2010-01-23,1,Saloon,Silver,TOYOTA,Yaba,Benue,Car Classic,0
4,ID_03U3I31,2010-09-06,2011-09-05,Male,39,2010-09-06,2,Saloon,Red,TOYOTA,Oshodi-Isolo,Oshodi-Isolo,Car Classic,0


In [43]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2667 entries, 0 to 2666
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   ID                      2667 non-null   object
 1   Policy Start Date       2667 non-null   object
 2   Policy End Date         2667 non-null   object
 3   Gender                  2667 non-null   object
 4   Age                     2667 non-null   int64 
 5   First Transaction Date  2667 non-null   object
 6   No_Pol                  2667 non-null   int64 
 7   Car_Category            2667 non-null   object
 8   Subject_Car_Colour      2667 non-null   object
 9   Subject_Car_Make        2667 non-null   object
 10  LGA_Name                2667 non-null   object
 11  State                   2667 non-null   object
 12  ProductName             2667 non-null   object
 13  target                  2667 non-null   int64 
dtypes: int64(3), object(11)
memory usage: 291.8+ KB


In [44]:
train.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Age,2667.0,45.110611,18.732415,-27.0,36.0,42.0,51.0,320.0
No_Pol,2667.0,1.499063,0.805725,1.0,1.0,1.0,2.0,7.0
target,2667.0,0.187477,0.390367,0.0,0.0,0.0,0.0,1.0


In [45]:
for col in train.columns:
  print(f'The number of unique values of {col}: {len(train[col].unique())}')

The number of unique values of ID: 2667
The number of unique values of Policy Start Date: 344
The number of unique values of Policy End Date: 350
The number of unique values of Gender: 6
The number of unique values of Age: 88
The number of unique values of First Transaction Date: 344
The number of unique values of No_Pol: 7
The number of unique values of Car_Category: 14
The number of unique values of Subject_Car_Colour: 34
The number of unique values of Subject_Car_Make: 45
The number of unique values of LGA_Name: 202
The number of unique values of State: 88
The number of unique values of ProductName: 9
The number of unique values of target: 2


3.1). Dealing with age

In [46]:
train[train['Age']<0]

,ID,Policy Start Date,Policy End Date,Gender,Age,First Transaction Date,No_Pol,Car_Category,Subject_Car_Colour,Subject_Car_Make,LGA_Name,State,ProductName,target
255,ID_3QOZW9J,2010-03-15,2011-03-14,Male,-2,2010-03-15,1,Saloon,Black,TOYOTA,Ibadan South East,Ibadan-East,Customized Motor,0
569,ID_7OMERTL,2010-04-23,2011-04-22,Male,-12,2010-04-23,3,Saloon,As Attached,Hyundai,Victoria Island,Lagos,Car Plus,0
1668,ID_N21XK5R,2010-04-23,2011-04-22,Male,-27,2010-04-23,2,JEEP,Blue,TOYOTA,Ilesha West,Ilesha-West,Car Classic,1


In [47]:
train['Age']=train['Age'].abs()

In [48]:
train.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Age,2667.0,45.141357,18.658173,0.0,36.0,42.0,51.0,320.0
No_Pol,2667.0,1.499063,0.805725,1.0,1.0,1.0,2.0,7.0
target,2667.0,0.187477,0.390367,0.0,0.0,0.0,0.0,1.0


In [49]:
train[train['Age']>100]

,ID,Policy Start Date,Policy End Date,Gender,Age,First Transaction Date,No_Pol,Car_Category,Subject_Car_Colour,Subject_Car_Make,LGA_Name,State,ProductName,target
21,ID_0BNN4SR,2010-07-31,2011-07-30,Female,120,2010-07-31,1,Saloon,Black,TOYOTA,Victoria Island,Lagos,Car Classic,0
64,ID_13D8KE6,2010-05-16,2011-05-15,Entity,120,2010-05-16,2,JEEP,Black,TOYOTA,Victoria Island,Lagos,Car Classic,0
207,ID_2WEXEBY,2010-04-18,2011-04-17,Male,120,2010-04-18,1,Bus,White,Scania,Victoria Island,Lagos,CVTP,0
235,ID_3GMJR1Z,2010-05-14,2011-05-13,Entity,120,2010-05-14,3,Saloon,Silver,TOYOTA,Lagos Island,Lagos,Car Classic,0
265,ID_3U7GP0U,2010-07-02,2011-07-01,Male,120,2010-07-02,1,Truck,Red,Iveco,Victoria Island,Lagos,CVTP,1
419,ID_5WP561N,2010-07-23,2011-07-22,Male,120,2010-07-23,1,Bus,White,Volkswagen,Victoria Island,Lagos,CVTP,0
474,ID_6O67NWQ,2010-06-28,2011-06-27,Male,120,2010-06-28,1,Truck,White,Iveco,Victoria Island,Lagos,CVTP,0
518,ID_76G0BAZ,2010-05-02,2011-05-01,Male,120,2010-05-02,1,Truck,Blue,Iveco,Victoria Island,Lagos,CVTP,1
564,ID_7NHCEJS,2010-07-03,2011-07-02,Male,120,2010-07-03,1,Saloon,Silver,Honda,Abuja,Abuja,Customized Motor,0
594,ID_7YCA0E9,2010-05-16,2011-05-15,Entity,120,2010-05-16,2,JEEP,Black,TOYOTA,Victoria Island,Lagos,Car Classic,1


In [50]:
#It seems a common error on 320years
#we replace 320 to 32
train['Age'].replace({320:32}, inplace=True)

In [51]:
train.Age.describe().transpose()

count    2667.000000
mean       44.925384
std        17.074416
min         0.000000
25%        36.000000
50%        42.000000
75%        51.000000
max       120.000000
Name: Age, dtype: float64

3.2). Dealing with LGA_Names and state

In [52]:
train[(train.LGA_Name.isnull()) & (~train.State.isnull())]

,ID,Policy Start Date,Policy End Date,Gender,Age,First Transaction Date,No_Pol,Car_Category,Subject_Car_Colour,Subject_Car_Make,LGA_Name,State,ProductName,target


In [53]:
StateName['LGA'] = StateName.LGA.str.replace('--',' ').str.replace('-',' ').str.title()
train['LGA_Name'] = train.LGA_Name.str.replace('-',' ').str.replace('/',' ').str.title()
train['LGA_Name'] = train.LGA_Name.str.replace('.',' ').str.replace(',','').str.strip()

In [54]:
LGA_not_found = []
for lga_name in train.LGA_Name:
  if lga_name not in StateName.LGA.unique():
    LGA_not_found.append(lga_name)

len(list(set(LGA_not_found)))

11

In [55]:

list(set(LGA_not_found))

['Olamabolo',
 'Ogba',
 'Ifako Ijaye',
 'Bekwara',
 'Ogudu',
 'Obia Akpor',
 'Somolu',
 'Ogbmosho South',
 'AjegunleLagos  State',
 'Ilesha West',
 'Ovia Southwest']

In [56]:
for lga in StateName.LGA.unique():
  if  (type(lga) != float):
    if ('Shomolu' in lga):
      print(lga)

Shomolu


In [57]:

correct_LGA = {
               'Obia Akpor':'Obio Akpor',
               'Ogbmosho South':'Ogbomosho South',
               'Ovia Southwest':'Ovia South West',
               'Ovia Southwest':'Ovia South West',
               'Ilesha East':'Ilesa East',
               'Ilesha West':'Ilesa West',
               'Ogudu':'Ojota',
               'Olamabolo':'Olamaboro',
                'Calabar Municipality':'Calabar Municipal',
               'Calabar Municipality':'Calabar Municipal',
               'Ifako Ijaye':'Ifako Ijaiye',
               'AjegunleLagos  State':'Ajegunle Lagos State',
               'Bekwara':'Bekwarra',
               'Ogba':'Ogbadibo',
               'Somolu':'Shomolu'
               }

train['LGA_Name'] = train['LGA_Name'].replace(correct_LGA)

In [58]:
train['State'] = train['LGA_Name'].replace(dict(zip(StateName.LGA,StateName.State)))

3.3. Dealing With Subject_Car_Colour

In [59]:
train.Subject_Car_Colour.value_counts().to_frame()

,Subject_Car_Colour
Black,1147
Silver,264
Grey,258
Blue,174
As Attached,165
Green,153
White,143
Red,125
Gold,74
Ash,43


In [60]:
modified_colors = {'Ash':'Grey',
                   'Wine':'Red',
                   'Cream':'White',
                   'Champagne':'Beige',
                   'Burgundy':'Red'}

train['Subject_Car_Colour'] = train.Subject_Car_Colour.replace(modified_colors)

In [61]:
train['Gender'].value_counts()

Male            1673
Female           820
Entity            91
Joint Gender      56
NO GENDER         20
NOT STATED         7
Name: Gender, dtype: int64

In [62]:
train['Gender'] = train['Gender'].replace({'Entity': 'Other',
                                                'Joint Gender':'Other',
                                                'NOT STATED':'Other',
                                                'NO GENDER': 'Other',
                                                'SEX':'Other'})
train['Gender'].value_counts()

Male      1673
Female     820
Other      174
Name: Gender, dtype: int64

4. Building Model

In [26]:
train=train[['Gender','Age','No_Pol','Car_Category','Subject_Car_Colour', 'Subject_Car_Make', 'LGA_Name', 'State','ProductName', 'target']]
train.head()

,Gender,Age,No_Pol,Car_Category,Subject_Car_Colour,Subject_Car_Make,LGA_Name,State,ProductName,target
0,Female,71,2,JEEP,Black,REXTON,Badagry,Lagos State,Car Classic,0
1,Male,30,1,Saloon,Grey,TOYOTA,Ikeja,Lagos State,Car Classic,1
2,Male,40,2,Saloon,Black,Honda,Abuja Municipal,Abuja-Municipal-Area-Council,Car Classic,0
3,Male,41,1,Saloon,Silver,TOYOTA,Yaba,Lagos-Mainland,Car Classic,0
4,Male,39,2,Saloon,Red,TOYOTA,Oshodi Isolo,Lagos State,Car Classic,0


In [71]:
train['ProductName'].unique()

array(['Car Classic', 'Car Plus', 'CVTP', 'Customized Motor', 'CarFlex',
       'CarSafe', 'Motor Cycle', 'Muuve', 'Car Vintage'], dtype=object)

In [27]:
le_gender=LabelEncoder()
train['Gender']=le_gender.fit_transform(train['Gender'])
le_category=LabelEncoder()
train['Car_Category']=le_category.fit_transform(train['Car_Category'])
le_color=LabelEncoder()
train['Subject_Car_Colour']=le_color.fit_transform(train['Subject_Car_Colour'])
le_make=LabelEncoder()
train['Subject_Car_Make']=le_make.fit_transform(train['Subject_Car_Make'])

le_lga=LabelEncoder()
train['LGA_Name']=le_category.fit_transform(train['LGA_Name'])
le_state=LabelEncoder()
train['State']=le_color.fit_transform(train['State'])
le_product=LabelEncoder()
train['ProductName']=le_make.fit_transform(train['ProductName'])

In [28]:
train.head()

,Gender,Age,No_Pol,Car_Category,Subject_Car_Colour,Subject_Car_Make,LGA_Name,State,ProductName,target
0,0,71,2,1,4,34,38,51,1,0
1,1,30,1,7,16,41,85,51,1,1
2,1,40,2,7,4,13,6,1,1,0
3,1,41,1,7,24,41,196,52,1,0
4,1,39,2,7,22,41,163,51,1,0


In [29]:
X=train.drop('target',axis=1)
y=train['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [30]:
w = 0.2
model = LGBMClassifier(n_estimators=800,
                       num_leaves=20,
                       min_child_samples=11,
                        class_weight={0:w,1:1-w})

In [31]:
model.fit(X_train,y_train)
pred=model.predict(X_test)
f1_score(pred,y_test)

0.28938906752411575

In [32]:
X.columns

Index(['Gender', 'Age', 'No_Pol', 'Car_Category', 'Subject_Car_Colour',
       'Subject_Car_Make', 'LGA_Name', 'State', 'ProductName'],
      dtype='object')

X=np.array([['Male',25,1,'JEEP','black','Toyota','Ikeja','Lagos State','Car Classic']])
X[:,0]=le_gender.transform(X[:,0])
X[:,3]=le_category.transform(X[:,3])
X[:,4]=le_color.transform(X[:,4])
X[:,5]=le_make.transform(X[:,5])
X[:,6]=le_lga.transform(X[:,6])
X[:,7]=le_state.transform(X[:,7])
X[:,8]=le_product.transform(X[:,8])

In [33]:
import pickle
data={'model':model,'le_gender':le_gender,'le_category':le_category,'le_color':le_color,'le_make':le_make,'le_lga':le_lga,'le_state':le_state,'le_product':le_product}
with open ('saved_steps.pkl','wb') as file:
  pickle.dump(data,file)

In [34]:
with open ('saved_steps.pkl','rb') as file:
  saved_data=pickle.load(file)

In [35]:
saved_data['model'].predict(X)

array([0, 1, 0, ..., 0, 0, 0], dtype=int64)

In [36]:
dd.drop('target',axis=1,inplace=True)

In [38]:
#dd['Gender'] = saved_data['le_gender'].transform(dd['Gender'])
#dd['Car_Category'] = saved_data['le_category'].fit_transform(dd['Car_Category'])
dd['Subject_Car_Colour'] = saved_data['le_color'].fit_transform(dd['Subject_Car_Colour'])

dd['Subject_Car_Make'] = saved_data['le_make'].fit_transform(dd['Subject_Car_Make'])
dd['State'] = saved_data['le_state'].fit_transform(dd['State'])
dd['LGA_Name'] = saved_data['le_lga'].fit_transform(dd['LGA_Name'])
dd['ProductName'] = saved_data['le_product'].fit_transform(dd['ProductName'])

In [39]:
dd['Car_Category'] = le_category.fit_transform(dd['Car_Category'])

In [40]:
#dd['Subject_Car_Make'] = le_make.transform(dd['Subject_Car_Make'])
dd['State'] = le_state.transform(dd['State'])
dd['LGA_Name'] = le_lga.transform(dd['LGA_Name'])
dd['ProductName'] = le_product.transform(dd['ProductName'])

NotFittedError: This LabelEncoder instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [1]:
l

NameError: name 'l' is not defined